Start by importing a load of stuff

Add to this later if need more packages

In [1]:
import sympy as sym
sym.init_printing()

import numpy as np
from math import pi
import math

import matplotlib.pyplot as plt
from sympy import I, Matrix, symbols
from sympy.physics.quantum import TensorProduct, Dagger
import scipy.optimize
import scipy.integrate

import scipy.constants as const

#import qutip

from matplotlib.colors import Normalize as Norm

%load_ext cython
%matplotlib notebook

In [2]:
#define some s pre/post operators
#used for defining superoperators like L in matrix form

def spre(m):
    return TensorProduct(sym.eye(m.shape[0]),m)

def spost(m):
    return TensorProduct(m.T, sym.eye(m.shape[0]))

def collapse(c):
    tmp = Dagger(c)*c/2
    return spre(c)*spost(Dagger(c))-spre(tmp)-spost(tmp)


s13=Matrix([[0,0,1],[0,0,0],[0,0,0]])
s23=Matrix([[0,0,0],[0,0,1],[0,0,0]])
s12=Matrix([[0,1,0],[0,0,0],[0,0,0]])

s31=s13.T
s32=s23.T
s21=s12.T

s11 = s12*s21
s22 = s21*s12
s33 = s31*s13

In [3]:
#define symbols that are used for the symbolic stuff
delta2,delta3=sym.symbols('delta_2 delta_3', real=True)
gamma13,gamma23,gamma2d,gamma3d,nbath,gammamu=sym.symbols('gamma_13 gamma_23 gamma_2d gamma_3d n_b gamma_mu', real=True, negative=False)
omegao, omegam=sym.symbols('Omega_o Omega_mu', real=True, negative=False)
rho11, rho12, rho13, rho21, rho22, rho23, rho31, rho32, rho33=sym.symbols('rho_11 rho_12 rho_13 rho_21 rho_22 rho_23 rho_31 rho_32 rho_33')

a= sym.symbols('a')
ar,ai=sym.symbols('a_r a_i', real=True)
g=sym.symbols('g',real=True, negative=False)
lam=sym.symbols('lambda')

In [4]:
#define the Hamiltonian for the system
#might add to this if we want to include more stuff
H=omegam*s21+omegao*s32+ g*a*s31
H=H+Dagger(H)
H=H+delta2*s22+delta3*s33

H_no_a=H[:,:]
H_no_a[0,2]=0
H_no_a[2,0]=0
H_no_a

H_o=H_no_a[:,:]
H_o[0,1]=0
H_o[1,0]=0

H_mu=H_no_a[:,:]
H_mu[2,1]=0
H_mu[1,2]=0


H, H_no_a, H_o, H_mu

⎛⎡            _⎤                                             ⎞
⎜⎢ 0   Ωₘᵤ  g⋅a⎥  ⎡ 0   Ωₘᵤ  0 ⎤  ⎡0  0   0 ⎤  ⎡ 0   Ωₘᵤ  0 ⎤⎟
⎜⎢             ⎥  ⎢            ⎥  ⎢         ⎥  ⎢            ⎥⎟
⎜⎢Ωₘᵤ  δ₂   Ωₒ ⎥, ⎢Ωₘᵤ  δ₂   Ωₒ⎥, ⎢0  δ₂  Ωₒ⎥, ⎢Ωₘᵤ  δ₂   0 ⎥⎟
⎜⎢             ⎥  ⎢            ⎥  ⎢         ⎥  ⎢            ⎥⎟
⎝⎣a⋅g  Ωₒ   δ₃ ⎦  ⎣ 0   Ωₒ   δ₃⎦  ⎣0  Ωₒ  δ₃⎦  ⎣ 0    0   δ₃⎦⎠

In [117]:
#define the Liovillian for the system in Matrix form
#We change the first row to impose the normalisation condition
LH=-I*spre(H)+I*spost(H)
L21 = gammamu*(nbath+1)*collapse(s12)
L12 = gammamu*nbath*collapse(s21)
L32 = gamma23*collapse(s23)
L31 = gamma13*collapse(s13)
L22 = gamma2d*collapse(s22)
L33 = gamma3d*collapse(s33)

L = LH + L21 + L12 + L32 + L31 + L22 + L33
L0=L.subs(a,0)
L1=L-L0
#L0 = L0.row_insert(0,Matrix([[1,0,0,0,1,0,0,0,1]]))
#L0.row_del(1)

In [118]:
L0

⎡-γₘᵤ⋅n_b                  -ⅈ⋅Ωₘᵤ                                  0          
⎢                                                                             
⎢                  γ_2d   γₘᵤ⋅n_b   γₘᵤ⋅(n_b + 1)                             
⎢ -ⅈ⋅Ωₘᵤ   -ⅈ⋅δ₂ - ──── - ─────── - ─────────────                -ⅈ⋅Ωₒ        
⎢                   2        2            2                                   
⎢                                                                             
⎢                                                          γ₁₃   γ₂₃   γ_3d   
⎢   0                      -ⅈ⋅Ωₒ                   -ⅈ⋅δ₃ - ─── - ─── - ──── - 
⎢                                                           2     2     2     
⎢                                                                             
⎢                                                                             
⎢ ⅈ⋅Ωₘᵤ                      0                                     0          
⎢                                                   

In [119]:
#define the density matrix in square and row form
#the row form is so the Liovillian in matrix form can be acted on it
rho = Matrix([[rho11,rho21,rho31],[rho12,rho22,rho32],[rho13,rho23,rho33]])
rho = 1*rho.T #because we are using "fortran" style matrix flatteneing
rho[:]
rhoflat = 1*rho.T 
rhoflat = rhoflat[:]

In [120]:
L1

⎡                    _                                          ⎤
⎢  0       0    -ⅈ⋅g⋅a    0     0    0     ⅈ⋅a⋅g     0      0   ⎥
⎢                                                               ⎥
⎢  0       0      0       0     0    0       0     ⅈ⋅a⋅g    0   ⎥
⎢                                                               ⎥
⎢-ⅈ⋅a⋅g    0      0       0     0    0       0       0    ⅈ⋅a⋅g ⎥
⎢                                                               ⎥
⎢                                       _                       ⎥
⎢  0       0      0       0     0  -ⅈ⋅g⋅a    0       0      0   ⎥
⎢                                                               ⎥
⎢  0       0      0       0     0    0       0       0      0   ⎥
⎢                                                               ⎥
⎢  0       0      0     -ⅈ⋅a⋅g  0    0       0       0      0   ⎥
⎢                                                               ⎥
⎢    _                                                         _⎥
⎢ⅈ⋅g⋅a    

In [121]:
H

⎡            _⎤
⎢ 0   Ωₘᵤ  g⋅a⎥
⎢             ⎥
⎢Ωₘᵤ  δ₂   Ωₒ ⎥
⎢             ⎥
⎣a⋅g  Ωₒ   δ₃ ⎦

In [122]:
#make lambda functions for the Liovillian so it can be used for numerics

Lfunc = sym.lambdify((a,delta2, delta3, gamma13, gamma23, gamma2d, gamma3d, nbath,gammamu,omegao,omegam,g),L)
L0func = sym.lambdify((delta2, delta3, gamma13, gamma23, gamma2d, gamma3d, nbath,gammamu,omegao,omegam),L0)
L1func = sym.lambdify((a,g),L1)
Hfunc = sym.lambdify((a,delta2,delta3,omegao, omegam,g),H)


In [123]:
def rho0_func(delta2val,delta3val,p):
    #aval=p['a']
    #delta2val=p['delta2']
    #delta3val=p['delta3']
    #a_r=np.real(a)
    #a_i=np.imag(a)
    L0_mat=L0func(delta2val,delta3val,p['gamma13'],p['gamma23'],p['gamma2d'],p['gamma3d'], p['nbath'],p['gammamu'],p['omegao'],p['omegam'])
    rho = np.linalg.solve(L0_mat.astype('complex128'),np.matrix([[1,0,0,0,0,0,0,0,0]]).T)
    return rho

In [124]:
# calculate microwave rabi frequency from power in in dBm
def omegam_from_Pin(Pin,p):
    mu0 = 4*pi*1e-7
    hbar=1.05457e-34; # in J*s
    Vsample=pi*((p['dsample']/2)**2) * p['Lsample'] # the volume of the Er:YSO sample
    V_microwave_cavity = Vsample/p['fillfactor']    
    mwP = 1e-3 * 10**(Pin/10)  #convert from dBm to Watts
    Q=2*pi*p['freqm']/(p['kappami']+p['kappamc']*2) # Q factor
    S21=(4*p['kappamc']**2)/(p['kappami']+p['kappamc']*2)**2
    energy_in_cavity=math.sqrt(S21)*2/(2*pi*p['freqm']/Q)*mwP; # energy inside the microwave cavity, in J
    Bmw=math.sqrt(mu0*(energy_in_cavity/V_microwave_cavity)/2); # Magnetic field of the microwave
    p['omegam']=(p['mu12']*Bmw)/hbar*(-1); # in rad/sec???? in Hz
    return p['omegam']
# calculate optical rabi frequency from power in in dBm
def omegao_from_Pin(Pin,p):
    epsilon0=8.854187817e-12
    hbar=1.05457e-34; # in J*s

    optP = 1e-3 * 10**(Pin/10) #incident optical power in W
    pflux = optP/(2*pi*p['freqo']*hbar) #photon flux (photons/sec)
    n_in = pflux * p['kappaoc']*4/(p['kappaoc']+p['kappaoi'])**2 # num intracavity photons
                                                #is this right????
    Sspot = pi*p['Wcavity']**2 #cross sectional area of
                                #optical mode
    V_cav = (Sspot*p['Lcavity_vac']+Sspot*p['Lsample']*p['nYSO']**3)/2;
    optEfield = math.sqrt(n_in*hbar*2*pi*p['freqo']/2/epsilon0/V_cav);
    p['omegao'] = p['d23']*optEfield/hbar*(-1);
    return p['omegao']

In [125]:
p = {}
p['a'] = 0
aval=p['a']
p['delta2'] = 0.
p['delta3'] = 0.


p['d13'] = 2e-32*math.sqrt(1/3)
p['d23'] = 2e-32*math.sqrt(2/3)
p['gamma13'] = p['d13']**2/(p['d13']**2+p['d23']**2)*1/11e-3
p['gamma23'] = p['d23']**2/(p['d13']**2+p['d23']**2)*1/11e-3
p['gamma2d'] = 1e6
p['gamma3d'] = 1e6
p['nbath'] = 20
p['gammamu'] = 1/(p['nbath']+1) * 1e3

p['g'] = 51.9  #optical coupling
p['N'] = 1.28e15 # number of atoms in the optical mode

p['deltac']=0 #detuning for 
p['kappaoi']=2*pi*7.95e6 # intrinsic loss for optical resonator
p['kappaoc']=2*pi*1.7e6 # coupling loss for optical resonator
p['df']=0.1e6 # how small descretisation step to take when integrating over the 
            # inhomogeneous lines

p['mean_delta2']=0
p['sd_delta2']=2*pi*25e6/2.355  #microwave inhomogeneous broadening
                                #2.355is to turn FWHM into standard deviation
p['mean_delta3']=0
p['sd_delta3']=2*pi*170e6/2.355 #optical inhomogeneous broadening

p['kappami'] = 650e3*2*pi # intrinsic loss for microwave cavity
p['kappamc'] = 70e3*2*pi  # coupling loss for optical cavity
                        # this is for one of the two output ports
p['Nmu'] = 2.22e16  #toal number of atoms
p['gmu'] = 1.04 #coupling between atoms and microwave field


muBohr=927.4009994e-26; # Bohr magneton in J/T in J* T^-1
p['mu12'] = 4.3803*muBohr # transition dipole moment for microwave cavity (J T^-1)

p['Lsample']=12e-3 # the length of the sample, in m
p['dsample']=5e-3 # the diameter of the sample, in m

p['fillfactor']=0.8 #microwave filling factor
p['freqm'] = 5.186e9
p['freqo'] = 195113.36e9

p['Lcavity_vac'] = 49.5e-3 # length of the vacuum part of the optical
                           # Fabry Perot (m)
p['Wcavity'] =  0.6e-3# width of optical resonator beam in sample (m)
p['nYSO'] = 1.76 #refractive index of YSO

# assume 1mW (=0dBm) input to both optical and to microwave cavities
omegao_from_Pin(0,p)
omegam_from_Pin(0,p)

#see what rabi freqs we have
#p['omegao']/1e6,p['omegam']/1e6


In [126]:
def rho_func(aval, delta2val,delta3val,p):
    xx=5
    L1_mat=L1func(aval,p['g'])
    L1_mat[xx,:]=[1,0,0,0,1,0,0,0,1]
    L0_mat=L0func(delta2val,delta3val,p['gamma13'],p['gamma23'],p['gamma2d'],p['gamma3d'], p['nbath'],p['gammamu'],p['omegao'],p['omegam'])
    rho0 = np.linalg.solve(L0_mat.astype('complex128'),np.matrix([[1,0,0,0,0,0,0,0,0]]).T)
    L1rho0=L1_mat*rho0
    L1rho0[xx]=0
    rho1 = np.linalg.solve(L0_mat.astype('complex128'),np.matrix(L1rho0))
    return rho0, rho1

In [314]:
def E_func(aval, delta2val, delta3val,p):
    H_mat=Hfunc(aval,delta2val,delta3val,p['omegao'],p['omegam'],p['g'])
    rho0,rho1=rho_func(aval,delta2val,delta3val,p)
    E0=np.trace(np.reshape(rho0,(3,3))*H_mat)
    E1=np.trace(np.reshape(rho1,(3,3))*H_mat)
    return E0, E1

In [322]:
E_func(10j,-10e6,40e6,p)

((6504716.984399124+1.4574878193229779e-08j),
 (-0.07419315766511708+2.5779378631796135e-13j))

In [317]:
p

{'a': 0,
 'delta2': 0.0,
 'delta3': 0.0,
 'd13': 1.1547005383792515e-32,
 'd23': 1.6329931618554522e-32,
 'gamma13': 30.303030303030297,
 'gamma23': 60.606060606060616,
 'gamma2d': 1000000.0,
 'gamma3d': 1000000.0,
 'nbath': 20,
 'gammamu': 47.61904761904761,
 'g': 51.9,
 'N': 1280000000000000.0,
 'deltac': 0,
 'kappaoi': 49951323.19207771,
 'kappaoc': 10681415.022205297,
 'df': 100000.0,
 'mean_delta2': 0,
 'sd_delta2': 66700480.96793615,
 'mean_delta3': 0,
 'sd_delta3': 453563270.5819659,
 'kappami': 4084070.449666731,
 'kappamc': 439822.971502571,
 'Nmu': 2.22e+16,
 'gmu': 1.04,
 'mu12': 4.06229459767182e-23,
 'Lsample': 0.012,
 'dsample': 0.005,
 'fillfactor': 0.8,
 'freqm': 5186000000.0,
 'freqo': 195113360000000.0,
 'Lcavity_vac': 0.0495,
 'Wcavity': 0.0006,
 'nYSO': 1.76,
 'omegao': -492090.88755145477,
 'omegam': -4754308.429163911}

In [243]:
L0_mat=L0func(10e6,40e6,p['gamma13'],p['gamma23'],p['gamma2d'],p['gamma3d'], p['nbath'],p['gammamu'],p['omegao'],p['omegam'])
L0_new=np.concatenate(([[1,0,0,0,1,0,0,0,1]],L0_mat),axis=0)
L0_new_pinv=np.linalg.pinv(L0_new)
#rho_new=L0_new_pinv*(np.array([[1,0,0,0,0,0,0,0,0,0]]))
np.matrix(L0_new_pinv)*np.matrix(np.array([[1,0,0,0,0,0,0,0,0,0]]).T)

matrix([[ 3.49634097e-01-6.53827278e-11j],
        [ 5.82006149e-05+2.69584271e-06j],
        [-9.57455326e-05-4.46806713e-06j],
        [ 5.82006149e-05-2.69584273e-06j],
        [ 3.49501487e-01-6.53587848e-11j],
        [ 8.12041902e-04+2.77909133e-05j],
        [-9.57455326e-05+4.46806717e-06j],
        [ 8.12041902e-04-2.77909136e-05j],
        [ 3.00864416e-01-5.62041265e-11j]])

In [248]:
L0_1 = L0.row_insert(0,Matrix([[1,0,0,0,1,0,0,0,1]]))
L0_1.row_del(1)

In [267]:
L0_1func = sym.lambdify((delta2, delta3, gamma13, gamma23, gamma2d, gamma3d, nbath,gammamu,omegao,omegam),L0_1)
L0_1_mat=L0_new1func(10e6,40e6,p['gamma13'],p['gamma23'],p['gamma2d'],p['gamma3d'], p['nbath'],p['gammamu'],p['omegao'],p['omegam'])
rho0=np.linalg.solve(L0_1_mat.astype('complex128'),np.matrix([[1,0,0,0,0,0,0,0,0]]).T)
rho0

matrix([[ 3.49634097e-01-7.23867639e-17j],
        [ 5.82006149e-05+2.69584272e-06j],
        [-9.57455326e-05-4.46806715e-06j],
        [ 5.82006149e-05-2.69584272e-06j],
        [ 3.49501487e-01-7.24423415e-17j],
        [ 8.12041902e-04+2.77909135e-05j],
        [-9.57455326e-05+4.46806715e-06j],
        [ 8.12041902e-04-2.77909135e-05j],
        [ 3.00864416e-01+1.45012886e-16j]])

In [279]:
L1rho0=L1func(10,p['g'])*rho0
L1rho0_new=np.concatenate(([[0]],L1rho0),axis=0)
#L1_new_pinv=np.linalg.pinv(L1_new)
#rho1_new=np.concatenate(([[0]],rho0),axis=0)
-L0_new_pinv*L1rho0_new

matrix([[-4.97867239e-06+1.34897199e-15j],
        [ 2.62467262e-08-4.84024827e-10j],
        [-6.14608172e-07-7.24611896e-09j],
        [ 2.62467262e-08+4.84024827e-10j],
        [-5.00879625e-06+1.34848517e-15j],
        [-1.49463423e-07-3.78985071e-09j],
        [-6.14608172e-07+7.24611896e-09j],
        [-1.49463423e-07+3.78985072e-09j],
        [ 9.98746864e-06+1.16053990e-15j]])

In [280]:
def rho_func_test(aval, delta2val,delta3val,p,rho0,xx):
    #xx=1
    L1_mat=L1func(aval,p['g'])
    L1_mat[xx,:]=[1,0,0,0,1,0,0,0,1]
    L0_mat=L0func(delta2val,delta3val,p['gamma13'],p['gamma23'],p['gamma2d'],p['gamma3d'], p['nbath'],p['gammamu'],p['omegao'],p['omegam'])
    #L0_mat1=L0_mat[:,:]
    #L0_mat1[0,:]=[1,0,0,0,1,0,0,0,1]
    L0_mat[xx,:]=[1,0,0,0,1,0,0,0,1]
    #rho0 = np.linalg.solve(L0_mat1.astype('complex128'),np.matrix([[1,0,0,0,0,0,0,0,0]]).T)
    L1rho0=L1_mat*rho0
    L1rho0[xx]=0
    rho1 = np.linalg.solve(L0_mat.astype('complex128'),-np.matrix(L1rho0)).T
    return rho0, rho1

In [297]:
def rho_func(aval, delta2val,delta3val,p):
    L0_mat=L0func(delta2val,delta3val,p['gamma13'],p['gamma23'],p['gamma2d'],p['gamma3d'], p['nbath'],p['gammamu'],p['omegao'],p['omegam'])
    L1_mat=L1func(aval,p['g'])
    L0_mat[0,:]=[1,0,0,0,1,0,0,0,1] #for replacing a row, have to replace row corresponding to population - rho_11, or 22, or 33
    rho0 = np.linalg.solve(L0_mat.astype('complex128'),np.matrix([[1,0,0,0,0,0,0,0,0]]).T)
    L1rho0=L1_mat*rho0
    L1rho0[0]=0
    rho1 = np.linalg.solve(L0_mat.astype('complex128'),-np.matrix(L1rho0))
    
    return rho0, rho1
    

In [301]:
rho_func(10,10e6,40e6,p)

(matrix([[ 3.49634097e-01-7.23867639e-17j],
         [ 5.82006149e-05+2.69584272e-06j],
         [-9.57455326e-05-4.46806715e-06j],
         [ 5.82006149e-05-2.69584272e-06j],
         [ 3.49501487e-01-7.24423415e-17j],
         [ 8.12041902e-04+2.77909135e-05j],
         [-9.57455326e-05+4.46806715e-06j],
         [ 8.12041902e-04-2.77909135e-05j],
         [ 3.00864416e-01+1.45012886e-16j]]),
 matrix([[-4.97867239e-06+5.76204158e-21j],
         [ 2.62467262e-08-4.84024827e-10j],
         [-6.14608172e-07-7.24611896e-09j],
         [ 2.62467262e-08+4.84024827e-10j],
         [-5.00879625e-06+5.50261617e-21j],
         [-1.49463423e-07-3.78985072e-09j],
         [-6.14608172e-07+7.24611896e-09j],
         [-1.49463423e-07+3.78985072e-09j],
         [ 9.98746864e-06-1.12491794e-20j]]))

In [323]:
rho1=(np.reshape(rho1,(3,3)))

In [324]:
rho0=(np.reshape(rho0,(3,3)))

In [312]:
H_mat=Hfunc(10,10e6,40e6,p['omegao'],p['omegam'],p['g'])
np.trace(H_mat*np.reshape(rho1,(3,3)))

(349.30767418231346-1.3454745130536683e-12j)

In [313]:
np.trace(H_mat*np.reshape(rho0,(3,3)))

(15528238.798715739+5.0792188233117486e-09j)

In [334]:
rho1/rho0

matrix([[-1.42396649e-05+5.23199651e-20j,
          4.49620023e-04-2.91428140e-05j,
          6.40875905e-03-2.23390546e-04j],
        [ 4.49620023e-04+2.91428140e-05j,
         -1.43312588e-05+5.15650719e-20j,
         -1.84002967e-04+1.63016194e-06j],
        [ 6.40875905e-03+2.23390546e-04j,
         -1.84002967e-04-1.63016194e-06j,
          3.31959119e-05-1.43540701e-19j]])

In [332]:
rho0

matrix([[ 3.49634097e-01-7.23867639e-17j,
          5.82006149e-05+2.69584272e-06j,
         -9.57455326e-05-4.46806715e-06j],
        [ 5.82006149e-05-2.69584272e-06j,
          3.49501487e-01-7.24423415e-17j,
          8.12041902e-04+2.77909135e-05j],
        [-9.57455326e-05+4.46806715e-06j,
          8.12041902e-04-2.77909135e-05j,
          3.00864416e-01+1.45012886e-16j]])

In [336]:
np.matrix([[1,2],[3,4]])*np.matrix([[1,1],[2,2]])

matrix([[ 5,  5],
        [11, 11]])

In [341]:
E_func(100000,10e6,40e6,p)

((15527245.059471041+5.083713006115431e-09j),
 (3429286.7931755898-3.1123477128858212e-09j))

In [342]:
L

⎡                                                                     _       
⎢-γₘᵤ⋅n_b                  -ⅈ⋅Ωₘᵤ                                -ⅈ⋅g⋅a       
⎢                                                                             
⎢                  γ_2d   γₘᵤ⋅n_b   γₘᵤ⋅(n_b + 1)                             
⎢ -ⅈ⋅Ωₘᵤ   -ⅈ⋅δ₂ - ──── - ─────── - ─────────────                -ⅈ⋅Ωₒ        
⎢                   2        2            2                                   
⎢                                                                             
⎢                                                          γ₁₃   γ₂₃   γ_3d   
⎢ -ⅈ⋅a⋅g                   -ⅈ⋅Ωₒ                   -ⅈ⋅δ₃ - ─── - ─── - ──── - 
⎢                                                           2     2     2     
⎢                                                                             
⎢                                                                             
⎢ ⅈ⋅Ωₘᵤ                      0                      

In [343]:
H

⎡            _⎤
⎢ 0   Ωₘᵤ  g⋅a⎥
⎢             ⎥
⎢Ωₘᵤ  δ₂   Ωₒ ⎥
⎢             ⎥
⎣a⋅g  Ωₒ   δ₃ ⎦

In [359]:
s13*s32

⎡0  1  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [358]:
s11.pinv()

NotImplementedError: Rank-deficient matrices are not yet supported.

In [366]:
collapse(s21)*Matrix(rhoflat)+Matrix(rhoflat)

⎡    0    ⎤
⎢         ⎥
⎢   ρ₂₁   ⎥
⎢   ───   ⎥
⎢    2    ⎥
⎢         ⎥
⎢   ρ₃₁   ⎥
⎢   ───   ⎥
⎢    2    ⎥
⎢         ⎥
⎢   ρ₁₂   ⎥
⎢   ───   ⎥
⎢    2    ⎥
⎢         ⎥
⎢ρ₁₁ + ρ₂₂⎥
⎢         ⎥
⎢   ρ₃₂   ⎥
⎢         ⎥
⎢   ρ₁₃   ⎥
⎢   ───   ⎥
⎢    2    ⎥
⎢         ⎥
⎢   ρ₂₃   ⎥
⎢         ⎥
⎣   ρ₃₃   ⎦

In [365]:
collapse(s12)*Matrix(rhoflat)+Matrix(rhoflat)

⎡ρ₁₁ + ρ₂₂⎤
⎢         ⎥
⎢   ρ₂₁   ⎥
⎢   ───   ⎥
⎢    2    ⎥
⎢         ⎥
⎢   ρ₃₁   ⎥
⎢         ⎥
⎢   ρ₁₂   ⎥
⎢   ───   ⎥
⎢    2    ⎥
⎢         ⎥
⎢    0    ⎥
⎢         ⎥
⎢   ρ₃₂   ⎥
⎢   ───   ⎥
⎢    2    ⎥
⎢         ⎥
⎢   ρ₁₃   ⎥
⎢         ⎥
⎢   ρ₂₃   ⎥
⎢   ───   ⎥
⎢    2    ⎥
⎢         ⎥
⎣   ρ₃₃   ⎦

In [375]:
s21*Matrix([1,0,0])

⎡0⎤
⎢ ⎥
⎢1⎥
⎢ ⎥
⎣0⎦

In [380]:
np.matrix(L0_new_pinv)*np.matrix([1,0,0,0,0,0,0,0,0,0]).T

matrix([[ 3.49634097e-01-6.53827278e-11j],
        [ 5.82006149e-05+2.69584271e-06j],
        [-9.57455326e-05-4.46806713e-06j],
        [ 5.82006149e-05-2.69584273e-06j],
        [ 3.49501487e-01-6.53587848e-11j],
        [ 8.12041902e-04+2.77909133e-05j],
        [-9.57455326e-05+4.46806717e-06j],
        [ 8.12041902e-04-2.77909136e-05j],
        [ 3.00864416e-01-5.62041265e-11j]])

In [386]:
s12*s22

⎡0  1  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [390]:
s33*s12*s33

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦